In [1]:
import pandas
from rcd_defs import *
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline


Code cleaning

In [10]:
#load dataset
dataframe = pandas.read_csv("10_05_2021.csv", dtype={'work_postal':'str'})
dataframe['end_date'] = dataframe['end_date'].fillna(pandas.to_datetime('2021-11-01'))

#Replace under_29 by .under_29 to make it easier when sorting
dataframe['age_group'] = dataframe['age_group'].replace(to_replace='under_29', value='.under_29')

#Fill in na values for age_group
dataframe['age_group'] = dataframe['age_group'].fillna(dataframe['age_group'].value_counts().index[0])

#Replace missing values in event with unknown, working, or job/division change
event = 'event'
emp_id = 'emplid_sec'
end_date = 'end_date'
for i in range(len(dataframe)):
    if dataframe[event][i] == "" or pandas.isna(dataframe[event][i]):
        j = i + 1
        if dataframe[emp_id][j] == dataframe[emp_id][i] and j < len(dataframe):
            dataframe[event][i] = "job/division change"
        elif dataframe[end_date][i] == "" or pandas.isna(dataframe[end_date][i]):
            dataframe[event][i] = "working"
        else:
            dataframe[event][i] = "unknown"

#encode class values as integers
encoder = LabelEncoder()

age_col = dataframe["age_group"]
encoder.fit(age_col)
enc_age = encoder.transform(age_col)
dataframe["age_group"] = enc_age.astype(float)

job_col = dataframe["jobtitle"]
encoder.fit(job_col)
enc_job = encoder.transform(job_col)
dataframe["jobtitle"] = enc_job.astype(float)

event_col = dataframe["event"]
encoder.fit(event_col)
enc_event = encoder.transform(event_col)
dataframe["event"] = enc_event

dataset = dataframe.values
Y = dataset[:, event_]
X = dataset[:,[duration, comprate, jobtitle, age_group]].astype(float)

# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(Y)

/var/folders/mr/mwsr6_lj70d61ys4kkzgnhnh0000gn/T/ipykernel_7294/3375641937.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[event][i] = "unknown"
/var/folders/mr/mwsr6_lj70d61ys4kkzgnhnh0000gn/T/ipykernel_7294/3375641937.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[event][i] = "job/division change"


[3 2 3 ... 1 2 1]


ML Model

In [14]:



#define baseline model
def baseline_model():
    #create model
    model = Sequential()
    model.add(Dense(8, input_dim = 4, activation='relu'))
    model.add(Dense(4, activation='softmax'))
    #compile model
    model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
    return model

estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose =0)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%%(%.2f%%)" % (results.mean()*100,results.std()*100))

/var/folders/mr/mwsr6_lj70d61ys4kkzgnhnh0000gn/T/ipykernel_7294/3899244586.py:11: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose =0)
